In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.2.0


# 5.1 二维卷积层

卷积神经网络（convolutional neural network）是含有卷积层（convolutional layer）的神经网络。本章中介绍的卷积神经网络均使用最常见的**二维卷积层**。它有高和宽两个空间维度，常用来处理图像数据。本节中，我们将介绍简单形式的二维卷积层的工作原理。

## 5.1.1 二维互相关运算

虽然卷积层得名于**卷积**（convolution）运算，但我们通常在卷积层中使用更加直观的**互相关**（cross-correlation）运算。在二维卷积层中，**一个二维输入数组**和**一个二维核**（kernel）数组**通过互相关运算**输出一个二维数组。
我们用一个具体例子来解释二维互相关运算的含义。如图5.1所示，输入是一个高和宽均为3的二维数组。我们将该数组的形状记为$3 \times 3$或（3，3）。核数组的高和宽分别为2。该数组在卷积计算中又称卷积核或过滤器（filter）。卷积核窗口（又称卷积窗口）的形状取决于卷积核的高和宽，即$2 \times 2$。图5.1中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0\times0+1\times1+3\times2+4\times3=19$。

<div align=center>
<img  src="../img/chapter05/5.1_correlation.svg"/>
</div>
<div align=center>图 5.1 二维互相关运算</div>

在二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。图5.1中的输出数组高和宽分别为2，其中的4个元素由二维互相关运算得出：

$$
0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.\\
$$

下面我们将上述过程实现在`corr2d`函数里。它接受输入数组`X`与核数组`K`，并输出数组`Y`。

In [2]:
def corr2d(X,K):
    h,w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] -h + 1, X.shape[1] - w + 1)))  # 这里就是变换公式，没记清楚，后面回过头来看下。
#     （𝑛ℎ−𝑘ℎ+1)×(𝑛𝑤−𝑘𝑤+1)  h w分别为高和宽
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j].assign(tf.cast(tf.reduce_sum(X[i:i+h,j:j+w] * K), dtype=tf.float32))
    return Y

我们可以构造图5.1中的输入数组`X`、核数组`K`来验证二维互相关运算的输出。

In [5]:
X = tf.constant([[0,1,2],[3,4,5],[6,7,8]])
K = tf.constant([[0,1],[2,3]])
corr2d(X,K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

## 5.1.2 二维卷积层

二维卷积层将输入和卷积核做**互相关运算**，并**加上一个标量偏差**来得到输出。**卷积层的模型参数**包括了**卷积核**和**标量偏差**。在训练模型的时候，通常**我们先对卷积核随机初始化**，然后**不断迭代卷积核和偏差**。

下面基于`corr2d`函数来实现一个自定义的二维卷积层。在构造函数`__init__`里我们声明`weight`和`bias`这两个模型参数。前向计算函数`forward`则是直接调用`corr2d`函数再加上偏差。

In [6]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
        
    def build(self,kernal_size):
        self.w = self.add_weight(name='w', shape=kernal_size,
                                initializer = tf.random_normal_initializer())
        self.b = self.add_weight(name='b', shape=(1,), initializer=tf.random_normal_initializer())
        
    def call(self, inputs):
        return corr2d(inputs, self.w) + self.b

卷积窗口形状为$p \times q$的卷积层称为$p \times q$卷积层。同样，$p \times q$卷积或$p \times q$卷积核说明卷积核的高和宽分别为$p$和$q$。

## 5.1.3 图像中物体边缘检测

下面我们来看一个卷积层的简单应用：检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张$6\times 8$的图像（即高和宽分别为6像素和8像素的图像）。它中间4列为黑（0），其余为白（1）。

In [7]:
X = tf.Variable(tf.ones((6,8)))
X[:, 2:6].assign(tf.zeros(X[:,2:6].shape))   # assign 可以赋值
X

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

然后我们构造一个高和宽分别为1和2的卷积核`K`。当它与输入做互相关运算时，如果横向相邻元素相同，输出为0；否则输出为非0。

In [8]:
K = tf.constant([[1,-1]], dtype = tf.float32)
K

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1., -1.]], dtype=float32)>

下面将输入`X`和我们设计的卷积核`K`做互相关运算。可以看出，我们将从白到黑的边缘和从黑到白的边缘分别检测成了1和-1。其余部分的输出全是0。

In [9]:
Y = corr2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

In [ ]:
"""学习到的，其实就是两种边界之间的 变化，识别并拿到这种变化"""

由此，我们可以看出，卷积层可通过重复使用**卷积核有效地表征局部空间**。

## 5.1.4 通过数据学习核数组

最后我们来看一个例子，它使用**物体边缘检测**中的输入数据`X`和输出数据`Y`来学习我们构造的核数组`K`。我们首先构造一个卷积层，将其**卷积核初始化成随机数组**。接下来在每一次迭代中，我们使用平方误差来比较`Y`和卷积层的输出，然后计算梯度来更新权重。简单起见，这里的卷积层忽略了偏差。

虽然我们之前构造了`Conv2D`类，但由于`corr2d`使用了对单个元素赋值（`[i, j]=`）的操作因而无法自动求梯度。下面我们使用tf.keras.layers提供的`Conv2D`类来实现这个例子。

In [10]:
# 二维卷积层使用4维输入输出，格式为(样本, 高, 宽, 通道)，这里批量大小（批量中的样本数）和通道数均为1
X = tf.reshape(X, (1,6,8,1))   # 第一个占位为 批量大小，最后一个为通道数。
Y = tf.reshape(Y, (1,6,7,1))
Y

<tf.Tensor: shape=(1, 6, 7, 1), dtype=float32, numpy=
array([[[[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]],

        [[ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.],
         [-1.],
         [ 0.]]]], dtype=float32)>

In [32]:
# 构造一个输出通道数为1（将在“多输入通道和多输出通道”一节介绍通道），核数组形状是(1, 2)的二维卷积层
conv2d = tf.keras.layers.Conv2D(1, (1,2))
#input_shape = (samples, rows, cols, channels)
# Y = conv2d(X)
conv2d.weights

[]

In [30]:
Y_hat = conv2d(X)
conv2d.weights

[<tf.Variable 'conv2d_12/kernel:0' shape=(1, 2, 1, 1) dtype=float32, numpy=
 array([[[[ 0.9448573]],
 
         [[-1.0464596]]]], dtype=float32)>,
 <tf.Variable 'conv2d_12/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [33]:
Y_hat = conv2d(X)   # 这一步，对conv2d.weights 进行了初始化赋值
for i in range(10):
    with tf.GradientTape(watch_accessed_variables=False) as g:
#         print(conv2d.weights[0])
        g.watch(conv2d.weights[0])
        Y_hat = conv2d(X)
        l = (abs(Y_hat - Y)) ** 2
        dl = g.gradient(l, conv2d.weights[0])
        lr = 3e-2  # 0.03
        update = tf.multiply(lr, dl)
        update_weights = conv2d.get_weights()
        update_weights[0] = conv2d.weights[0] - update   # 这里卷积核里面的两个参数，都进行了更新
        conv2d.set_weights(update_weights)
        
        if (i+1) % 2 == 0:
            print('batch %d, loss %.3f' % (i+1, tf.reduce_sum(l)))

batch 2, loss 14.189
batch 4, loss 5.574
batch 6, loss 2.243
batch 8, loss 0.912
batch 10, loss 0.372


In [36]:
conv2d.get_weights()[0]

array([[[[ 0.93187594]],

        [[-1.0578007 ]]]], dtype=float32)

可以看到，10次迭代后误差已经降到了一个比较小的值。现在来看一下学习到的核数组。

In [37]:
tf.reshape(conv2d.get_weights()[0],(1,2))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.93187594, -1.0578007 ]], dtype=float32)>

可以看到，学到的核数组与我们之前定义的核数组`K`较接近。

## 5.1.6 特征图和感受野

二维卷积层输出的二维数组可以看作是输入在空间维度（宽和高）上某一级的表征，也叫**特征图**（feature map）。影响元素$x$的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做$x$的**感受野**（receptive field）。**以图5.1为例，输入中阴影部分的四个元素是输出中阴影部分元素的感受野**。我们将图5.1中形状为$2 \times 2$的输出记为$Y$，并考虑一个更深的卷积神经网络：将$Y$与另一个形状为$2 \times 2$的核数组做互相关运算，输出单个元素$z$。那么，$z$在$Y$上的感受野包括$Y$的全部四个元素，在输入上的感受野包括其中全部9个元素。可见，我们可以**通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征**。

我们常使用“元素”一词来描述数组或矩阵中的成员。在神经网络的术语中，这些元素也可称为“单元”。当含义明确时，本书不对这两个术语做严格区分。


## 小结

* 二维卷积层的核心计算是二维互相关运算。在最简单的形式下，它对二维输入数据和卷积核做互相关运算然后加上偏差。
* 我们可以设计卷积核来检测图像中的边缘。
* 我们可以通过数据来学习卷积核。

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_convolutional-neural-networks/conv-layer.html)

